# Preparação do ambiente

## Bibliotecas

In [1]:
import json
import pandas as pd
import re
import string

from os import path as osp
from pprint import pprint

## Constantes e funções auxiliares

In [2]:
NULL_STRING = '#NULLO#'
FILE_ANUNCIOS = '../datasets/anuncios.parquet'
FILE_ANUNCIOS_AMOSTRA = '../datasets/anuncios_amostra.parquet'

In [3]:
def flatten_matrix(matrix):
    return [item for row in matrix for item in row]

In [4]:
def extract_amazon_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split())

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

def extract_amazon_desc(row):
    desc = '\n'.join(row)
    desc = desc.replace(NULL_STRING,' ')
    desc = re.sub('\s+', ' ', desc)
    return desc

In [5]:
def extract_magalu_images(row):
    imagens = []
    imagens.append(row['Imagem'].split()[0])
    imagens.extend(row['Imagens'].split(','))

    imagens = [imagem for imagem in imagens if imagem.startswith('http')]
    if len(imagens) >= 3:
        imagens = imagens[:3]
    elif len(imagens) == 2:
        imagens.extend([''])
    elif len(imagens) == 1:
        imagens.extend(['',''])
    else:
        imagens.extend(['','',''])

    return pd.Series(imagens)

In [6]:
def format_issue(row,remove_punct=False):
    nome,desc,img0,img1,img2 = row
    # remove punctiation from nome and desc
    if remove_punct:
        nome = nome.translate(str.maketrans('', '', string.punctuation))
        desc = desc.translate(str.maketrans('', '', string.punctuation))
    
    img_css = '{max-width: 250px; height: auto;}'
    if len(img2) > 0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}! !{img_css}{img2}!'
    elif len(img1) >0:
        img_list = f'!{img_css}{img0}! !{img_css}{img1}!'
    else:
        img_list = f'!{img_css}{img0}!'
    issue = f'h1. {nome}\n\n{desc}\n\n{img_list}'
    issue = issue.replace(NULL_STRING,' ')
    # remova ponto-e-vírgula e aspas duplas para evitar a 
    # identificação incorreta de campos ao importar para o redmine
    issue = issue.replace(';',',')
    issue = issue.replace('"',"''")
    # remove multiple linebrakes and spaces
    issue = re.sub('\n+','\n\n',issue)
    issue = re.sub('\s+',' ',issue)
    return issue

# Carga e prepação dos dados

## Dados pré processados

In [7]:
if osp.exists(FILE_ANUNCIOS):
    df_anuncios = pd.read_parquet(FILE_ANUNCIOS)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


## Dados brutos

### Amazon

In [8]:
file_amazon_reindexed = '../datasets/amazon_reindexed.parquet'
df_amazon = pd.read_parquet(file_amazon_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Descrição_Principal', 'Imagem', 'Imagens' ]
df_amazon = df_amazon[columns_to_keep]

df_amazon['desc'] = df_amazon[['Descrição', 'Descrição_Principal']].apply(lambda row: extract_amazon_desc(row),axis=1)
# df_amazon['img_list'] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)
df_amazon[['img0','img1','img2']] = df_amazon[['Imagem', 'Imagens']].apply(lambda row: extract_amazon_images(row),axis=1)

columns_to_keep = ['Link', 'Nome', 'desc', 'img0','img1','img2']
df_amazon = df_amazon[columns_to_keep]
df_amazon.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']
df_amazon.head()

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


### Magalu

In [9]:
file_magalu_reindexed = '../datasets/magalu_redindexed.parquet.gzip'
df_magalu = pd.read_parquet(file_magalu_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_magalu = df_magalu[columns_to_keep]

df_magalu[['img0','img1','img2']] = df_magalu[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_magalu = df_magalu[columns_to_keep]
df_magalu.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_magalu.sample(5)

,link,nome,desc,img0,img1,img2
2812,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/ausculta...,,
32293,https://www.magazineluiza.com.br/celular-para-...,"Celular Para Idoso Positivo Android 11 Tela 4""...",Celular do Idoso 16gb Letras Grandes Tela 4' A...,https://a-static.mlcdn.com.br/90x90/celular-pa...,https://a-static.mlcdn.com.br/90x90/celular-pa...,https://a-static.mlcdn.com.br/90x90/celular-pa...
28885,https://www.magazineluiza.com.br/carregador-gt...,Carregador GT-S5830B Galaxy Original Samsung M...,Serve no modelo anunciado no titulo e em todos...,https://a-static.mlcdn.com.br/90x90/carregador...,https://a-static.mlcdn.com.br/90x90/carregador...,https://a-static.mlcdn.com.br/90x90/carregador...
1264,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/baby-cal...,,
12909,https://validate.perfdrive.com/ca4df1c7abf7ea2...,Pedimos desculpas pela inconveniência...,#NULLO#,https://a-static.mlcdn.com.br/280x210/bateria-...,,


### Mercado Livre

In [10]:
file_ml_reindexed = '../datasets/ml_redindexed.parquet.gzip'
df_ml = pd.read_parquet(file_ml_reindexed)

columns_to_keep = ['Link', 'Nome', 'Descrição', 'Imagem', 'Imagens' ]
df_ml = df_ml[columns_to_keep]

df_ml[['img0','img1','img2']] = df_ml[['Imagem', 'Imagens']].apply(lambda row: extract_magalu_images(row),axis=1)
columns_to_keep = ['Link', 'Nome', 'Descrição', 'img0','img1','img2']
df_ml = df_ml[columns_to_keep]
df_ml.columns = ['link', 'nome', 'desc', 'img0', 'img1', 'img2']

df_ml.sample(5)

,link,nome,desc,img0,img1,img2
67036,https://www.mercadolivre.com.br/drone-lyzrc-l9...,Drone LYZRC L900 Pro con bolso com dual câmera...,Atravessar fronteiras físicas e alcançar espaç...,https://http2.mlstatic.com/D_NQ_NP_758545-MLA4...,,
61084,https://produto.mercadolivre.com.br/MLB-445794...,Carregador Portátil Power Bank 10000 Personali...,***ATENÇÃO COMO PERSONALIZAR** Após realizar a...,https://http2.mlstatic.com/D_NQ_NP_729585-MLB7...,https://http2.mlstatic.com/D_NQ_NP_729585-MLB7...,
34774,https://produto.mercadolivre.com.br/MLB-342574...,"Transceptor De Rádio Bidirecional, Comunicação...",#NULLO#,https://http2.mlstatic.com/D_NQ_NP_961435-CBT5...,,
58215,https://produto.mercadolivre.com.br/MLB-414986...,Bateria Externa Sem Fio Azul iPhone 10000mah -...,Produto a Pronta entrega. Sobre este item A ma...,https://http2.mlstatic.com/D_NQ_NP_847941-MLB7...,,
67856,https://www.mercadolivre.com.br/fone-de-ouvido...,Fone De Ouvido Sem Fio Bluetooth 5.1 M10 Displ...,O fone de ouvido sem fio bluetooth M10 possui ...,https://http2.mlstatic.com/D_NQ_NP_998573-MLU7...,,


### União dos dados em único dataframe

In [11]:
df_anuncios = pd.concat([df_amazon, df_magalu, df_ml])
df_anuncios = df_anuncios.drop_duplicates(subset='nome')
df_anuncios = df_anuncios.reset_index(drop=True)
df_anuncios.to_parquet(FILE_ANUNCIOS)
df_anuncios.head(5)

,link,nome,desc,img0,img1,img2
0,https://www.amazon.com.br/Especiais-Cinematic-...,Filtro de Lente de Efeitos Especiais Blue Stre...,Especificações: Modelo aplicável: este filtro ...,https://m.media-amazon.com/images/I/61ZdffRDj2...,https://m.media-amazon.com/images/I/51DeP4IE2F...,https://m.media-amazon.com/images/I/41w7kz5r4P...
1,https://www.amazon.com.br/Motores-Escova-Conju...,"Motores Sem Escova, 4 Conjuntos de Motores Sem...",1. Modelo de uso: Este motor sem escova de dro...,https://m.media-amazon.com/images/I/51aZmDncic...,https://m.media-amazon.com/images/I/31mER3G37I...,https://m.media-amazon.com/images/I/31jk4g-fUt...
2,https://www.amazon.com.br/Armazenamento-Protet...,"Bolsa de Armazenamento Protetora para Drone, C...",1. Excelente proteção: o case rígido do drone ...,https://m.media-amazon.com/images/I/612uWGXrp7...,https://m.media-amazon.com/images/I/41JfSjIxxR...,https://m.media-amazon.com/images/I/41CVth1dqO...
3,https://www.amazon.com.br/EQUSS-V-strom-Motoci...,EQUSS Para DUCATI DL650 DL1000 V-strom DL 650 ...,1. Material: liga de alumínio [Super Grip] - D...,https://m.media-amazon.com/images/I/51-m8iNiuL...,https://m.media-amazon.com/images/I/31VLMQCsxU...,https://m.media-amazon.com/images/I/31WjqM+BqB...
4,https://www.amazon.com.br/remote-control-brush...,"remote control car brushless motor servo kit,r...",,https://m.media-amazon.com/images/I/61Hzfl2r75...,https://m.media-amazon.com/images/I/41P1kyyx1l...,https://m.media-amazon.com/images/I/41I9plec-T...


# Selecionar e preparar amostra

## Selecionar amostra

In [12]:
n_samples = 20000
df_samples_to_label = df_anuncios.iloc[:,1:].sample(n_samples,random_state=111)
df_samples_to_label.head()

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,,,
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,,,
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,,,
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_964769-MLB4...,,
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",https://http2.mlstatic.com/D_NQ_NP_916585-MLB5...,,


## Exportar para LabelStudio

In [13]:
df_label_studio = df_samples_to_label.copy()
df_label_studio.head(5)

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,,,
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,,,
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,,,
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,https://http2.mlstatic.com/D_NQ_NP_964769-MLB4...,,
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",https://http2.mlstatic.com/D_NQ_NP_916585-MLB5...,,


In [14]:
# Lista de imagens para baixar
images = []
images.extend(df_label_studio['img0'].to_list())
images.extend(df_label_studio['img1'].to_list())
images.extend(df_label_studio['img2'].to_list())
images = [f'wget {image}\n' for image in images if len(image)>0]
with open('images_to_download.sh','w') as f:
    f.writelines(images)
images[:5]

['wget https://http2.mlstatic.com/D_NQ_NP_964769-MLB47511585800_092021-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_916585-MLB50832387072_072022-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_933918-MLB73838529583_012024-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_939283-MLB71022987135_082023-F.webp\n',
 'wget https://http2.mlstatic.com/D_NQ_NP_762629-MLB71451841056_092023-F.webp\n']

In [15]:
# ajustar caminho das imagens (local storage)
for col in ['img0', 'img1', 'img2']:
    df_label_studio[col] = df_label_studio[col].apply(lambda x: f'/data/local-files/?d=images/{x.split("/")[-1]}' if len(x) > 0 else '/data/local-files/?d=images/none.jpg')
df_label_studio.head()

,nome,desc,img0,img1,img2
114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,/data/local-files/?d=images/D_NQ_NP_964769-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",/data/local-files/?d=images/D_NQ_NP_916585-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [16]:
if 'sample_id' not in df_label_studio.columns:
    print('Create sample_id column')
    df_label_studio = df_label_studio.reset_index()
    new_columns = ['sample_id', 'nome', 'desc', 'img0', 'img1', 'img2']
    df_label_studio.columns = new_columns
df_label_studio.head()

Create sample_id column


,sample_id,nome,desc,img0,img1,img2
0,114007,Homoyoyo Prateleira De Suporte 3 Peças Suporta...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
1,61311,KICHOUSE 2 Em 1 Transmissor Receptor De Áudio ...,,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
2,101392,"Drone Arm Spare Parts, Fine Workmanship Drone ...",,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
3,181146,Antena Transmissor Usb Dongle Ant+ Stick Zwift...,#NULLO#,/data/local-files/?d=images/D_NQ_NP_964769-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg
4,177324,Adaptador Receptor Bluetooth Usb P2 Música Cha...,"Seja bem vindo, somos a RevKing! Garantimos os...",/data/local-files/?d=images/D_NQ_NP_916585-MLB...,/data/local-files/?d=images/none.jpg,/data/local-files/?d=images/none.jpg


In [17]:
samples = df_label_studio.to_dict('records')
samples[0]
with open('../datasets/samples_to_label_studio.json','w') as f:
    json.dump(samples,f)
pprint(samples[0])

{'desc': ' ',
 'img0': '/data/local-files/?d=images/none.jpg',
 'img1': '/data/local-files/?d=images/none.jpg',
 'img2': '/data/local-files/?d=images/none.jpg',
 'nome': 'Homoyoyo Prateleira De Suporte 3 Peças Suporta Suporte Para Laptop '
         'Montagem Para Laptop Suporte De Mesa Para Laptop Suportes De '
         'Prateleira Para Monitor Prateleira Para Tv Montagem Em',
 'sample_id': 114007}


## Exportar para fiscaliza

In [18]:
fiscaliza_sample_size = 400

fiscaliza_last_issue = 0
format_title = lambda i: f'LABEL_PACP_2024_{str(i+1+fiscaliza_last_issue).zfill(8)}'

df_fiscaliza = df_samples_to_label.copy()
df_fiscaliza = df_fiscaliza.sample(fiscaliza_sample_size,random_state=1653)
df_fiscaliza = df_fiscaliza.reset_index(drop=True)

df_fiscaliza['Descrição'] = df_fiscaliza.apply(lambda row: format_issue(row),axis=1)
df_fiscaliza['Título'] = df_fiscaliza.index.map(format_title)

columns_to_keep = ['Título', 'Descrição']
df_fiscaliza = df_fiscaliza[columns_to_keep]


df_fiscaliza['Unidade Descentralizada'] = 'UO021'
df_fiscaliza['Data de início'] = '09/04/2024'
df_fiscaliza['Data limite'] = '31/05/2024'

fiscais = ['maxwelfreitas', 
           'costalo',
           'josias',
           'romildo',
           'williams',
           'maxwelfreitas', 
           'joaotavares', 
           'peluchi']

#specify number of rows in each chunk
n = len(fiscais)
n_chuncks = round(len(df_fiscaliza)/n)

#split DataFrame into chunks
list_chuncks = [df_fiscaliza[i:i+n_chuncks].index for i in range(0,len(df_fiscaliza),n_chuncks)]
for i,chunck in enumerate(list_chuncks):
    df_fiscaliza.loc[chunck,'Fiscal Responsável'] = fiscais[i]

file_samples_fiscaliza = '../datasets/samples_to_fiscaliza.csv'
df_fiscaliza.to_csv(file_samples_fiscaliza,index=False,sep=';')

df_fiscaliza

,Título,Descrição,Unidade Descentralizada,Data de início,Data limite,Fiscal Responsável
0,LABEL_PACP_2024_00000001,h1. Bediffer Protetor de tensão de drone RC de...,UO021,09/04/2024,31/05/2024,maxwelfreitas
1,LABEL_PACP_2024_00000002,h1. DEERC Drone with Camera for Adults and Kid...,UO021,09/04/2024,31/05/2024,maxwelfreitas
2,LABEL_PACP_2024_00000003,"h1. Cabeceira para cama box Solteiro 0,90 Canc...",UO021,09/04/2024,31/05/2024,maxwelfreitas
3,LABEL_PACP_2024_00000004,h1. Carregador Sem Fio Magsafe Para iPhone Wat...,UO021,09/04/2024,31/05/2024,maxwelfreitas
4,LABEL_PACP_2024_00000005,"h1. Carregador de painel solar, carregador sol...",UO021,09/04/2024,31/05/2024,maxwelfreitas
...,...,...,...,...,...,...
395,LABEL_PACP_2024_00000396,h1. Fone Xiaomi GT1 (PRO) Haylou TWS Sem fio B...,UO021,09/04/2024,31/05/2024,peluchi
396,LABEL_PACP_2024_00000397,h1. fone bluetooth y30 Acabou de chegar o melh...,UO021,09/04/2024,31/05/2024,peluchi
397,LABEL_PACP_2024_00000398,h1. Estabilizador gimbal de 3 eixos para smart...,UO021,09/04/2024,31/05/2024,peluchi
398,LABEL_PACP_2024_00000399,h1. Carregador Duplo De Bateria 18v Dc18rd 127...,UO021,09/04/2024,31/05/2024,peluchi
